### 查看环境版本

In [94]:
import sys
print(sys.version)
print(sys.executable)

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
C:\ProgramData\Anaconda3\python.exe


### 介绍：数据集中的第一列是用户的编号，第二列是电影的编号，第三列是用户对已观看过的电影的评分

### CF（基于用户的协同过滤算法 和 基于物品的协同过滤算法）
训练train.txt文件，给用户未观看过的电影预测评分，将预测的评分从大到小排序，选择预测评分最高的N=5部电影推荐给用户

In [95]:
import math

class CF:
    def __init__(self, datafile):
        self.datafile = datafile  # 传入文件名称 train.txt
        self.data = [] # 保存文件内容信息 # [(1,2,3), (1,14,5), ...] (用户，物品，评分)
        self.trainData = {} # 标准格式文件 # {1: {1: 4, 13: 4}, 2: {50: 5},...}  用户：{物品：评分}
        self.userSimMatrix = [] # 基于用户CF的推荐序列 {1: [28, 82, 127, 50, 4], 2: [7, 109, 118, 127, 117], ...}  用户：[物品]
        self.itemSimMatrix = [] # 基于物品CF的推荐序列 {1: [28, 82, 127, 50, 4], 2: [7, 109, 118, 127, 117], ...}  用户：[物品]

    def readData(self):
        """
        读取train.txt 和 test.txt 数据
        """
        datalist = []
        for line in open(self.datafile):
            userid, itemid, record = line.split(" ") # 用空格分割
            datalist.append((int(userid), int(itemid), int(record)))
        self.data = datalist

    def preprocessData(self):
        """
        把读入的数据转换为训练CF模型需要的格式
        """
        traindata_list = {}
        # 存储格式：
        for user, item, record in self.data:
            traindata_list.setdefault(user, {})
            traindata_list[user][item] = record
        self.trainData = traindata_list

    def userSimilarity(self):
        """
        生成用户相似度矩阵
        """
        self.userSimMatrix = dict()
        # 物品用户倒排表
        item_users = dict()
        for u, item in self.trainData.items():
            for i in item.keys():
                item_users.setdefault(i, set())
                item_users[i].add(u)
        # 计算用户间同时评分的物品
        user_item_count = dict()
        count = dict()
        for item, users in item_users.items():
            for u in users:
                user_item_count.setdefault(u, 0)
                user_item_count[u] += 1
                for v in users:
                    if u == v : continue
                    count.setdefault(u, {})
                    count[u].setdefault(v, 0)
                    count[u][v] += 1
        # 计算相似度矩阵
        for u, related_users in count.items():
            self.userSimMatrix.setdefault(u, dict())
            for v, cuv in related_users.items():
                self.userSimMatrix[u][v] = cuv / math.sqrt(user_item_count[u] * user_item_count[v] * 1.0)

    def userRecommend(self, user_id, k, N):
        '''
        给用户推荐K个与之相似用户喜欢的物品
        :param user: 用户id
        :param k: 近邻范围
        :param N: 推荐列表长度
        :return: 推荐列表
        '''
        rank = dict() # k个近邻用户的
        interacted_items = self.trainData.get(user_id, {}) # 当前用户已经交互过的item
        # 取最相似的k个用户的item
        # nbor_u是近邻用户的id，nbor_u_sim是近邻用户与当前用户的相似度
        for nbor_u, nbor_u_sim in sorted(self.userSimMatrix[user_id].items(), key=lambda x:x[1], reverse=True)[0:k]:
            for i, i_score in self.trainData[nbor_u].items(): # 取出所有近邻用户的item
                if i in interacted_items: # 不计入用户已经交互过的item
                    continue
                rank.setdefault(i, 0) # 初始化rank
                rank[i] += nbor_u_sim # 相似度求和，作为item的得分
        # 取出得分最高的N个item作为推荐列表
        return dict(sorted(rank.items(), key=lambda x:x[1], reverse=True)[0:N])

    
    def itemSimilarity(self):
        """
        生成物品相似度矩阵
        """
        # 物品相似度矩阵
        self.itemSimMatrix = dict()
        # 物品-物品矩阵 格式：物品ID1:{物品ID2:同时给两件物品评分的人数}
        item_item_matrix = dict()
        # 物品-用户矩阵 格式：物品ID:给物品评分的人数
        item_user_matrix = dict()

        for user, items in self.trainData.items():
            for itemId, source in items.items():
                item_user_matrix.setdefault(itemId, 0) # 初始化item_user_matrix[itemId]
                item_user_matrix[itemId] += 1 # 给物品打分的人数+1
                item_item_matrix.setdefault(itemId, {})
                for i in items.keys():
                    if i == itemId:
                        continue
                    item_item_matrix[itemId].setdefault(i, 0)
                    # 计算同时给两个物品打分的人数，并对活跃用户进行惩罚
                    item_item_matrix[itemId][i] += 1 / math.log(1 + len(items) * 1.0)
        # 将物品-物品矩阵转换为物品相似度矩阵
        for itemId, relatedItems in item_item_matrix.items():
            self.itemSimMatrix.setdefault(itemId, dict()) # 初始化self.itemSimMatrix[itemId]
            for relatedItemId, count in relatedItems.items():
                self.itemSimMatrix[itemId][relatedItemId] = count / math.sqrt(item_user_matrix[itemId] * item_user_matrix[relatedItemId])
            # 归一化
            sim_max = max(self.itemSimMatrix[itemId].values())
            for item in self.itemSimMatrix[itemId].keys():
                self.itemSimMatrix[itemId][item] /= sim_max

                
    def itemRecommend(self, user_id, k, N):
        """给用户推荐物品列表
            Args:
                userId:用户ID
                k:取和物品j最相似的K个物品
                N:推荐N个物品
            Return:
                推荐列表
            """
        rank = dict()
        interacted_items = self.trainData.get(user_id, {}) # 当前用户已经交互过的item
        # 遍历用户评分的物品列表
        for itemId, score in interacted_items.items(): # 取出每一个当前用户交互过的item
            # 取出与物品itemId最相似的K个物品及其评分
            for i, sim_ij in sorted(self.itemSimMatrix[itemId].items(), key=lambda x: x[1], reverse=True)[0:k]:
                # 如果这个物品j已经被用户评分了，舍弃
                if i in interacted_items.keys():
                    continue
                # 对物品ItemID的评分*物品itemId与j的相似度 之和
                rank.setdefault(i, 0)
                rank[i] += score * sim_ij
        # 堆排序，推荐权重前N的物品
        return dict(sorted(rank.items(), key=lambda x:x[1], reverse=True)[0:N])


### 评估

利用test.txt中的真实评分去评估所预测评分，选取的评价指标为Precision，Recall，F-measure

In [96]:
def FMeasure(rec_dict, val_dict):
    """
    rec_dict: 推荐列表或评分列表，形式为：{user_id:{item1, item2,....}, user_id:{item1, item2,....}}
    val_dict: 用户实际的点击列表或评分列表（测试集），形式为：{user_id:{item1, item2,....}, user_id:{item1, item2,....}}
    """
    # 推荐列表中用户点击的项目数
    hit_items = 0
    # 所有的项目
    allPre_items = 0 # 统计Precision
    allRec_items = 0 # 统计Recall
    
    for user_id, items in val_dict.items():
        # 测试集中真实的点击列表
        real_set = items
        # 推荐算法返回的推荐列表
        rec_set = rec_dict[user_id]
        # 当前用户推荐列表中有多少是实际点击的
        for item in rec_set:
            if item in real_set:
                hit_items += 1
        # 注意区别，Precision 统计推荐列表中的样本,Recall 统计测试集里的样本
        allPre_items += len(rec_set)
        allRec_items += len(real_set)
        
    pre = hit_items / allPre_items * 100 # 计算 Precision
    rec = hit_items / allRec_items * 100 # 计算 Recall
    F = (2 * pre * rec) / (pre + rec) # 计算 F-measure
    
    return round(pre, 2), round(rec, 2), round(F, 2)

### 运行训练集 和 测试集，并评估结果

In [97]:
if __name__ == "__main__":
    cf = CF('data/train.txt')
    cf.readData() # 读取数据
    cf.preprocessData() # 预处理数据
    cf.userSimilarity() # 计算用户相似度矩阵
    cf.itemSimilarity() # 计算物品相似度矩阵
   
    print('-----------# ----------- 基于【用户】为全部用户（打印id前5）产生推荐 ----------- #-----------')
    print()
    user_topN_list = {} # 存储为每一个用户推荐的列表
    for each_user in cf.trainData:
        user_topN = cf.userRecommend(each_user, k=3, N=5) # item的id和评分
        user_topN_list[each_user] = list(user_topN.keys()) # 只取对应的item的id
        if(each_user <= 5): # 输出id在1~5的用户推荐情况
            print('------ 长度5的电影序列user_topN_list的用户id：[{}] -----'.format(each_user))
            print(user_topN_list[each_user])
            print()

    print('-----------# ----------- 基于【物品】为全部用户（打印id前5）产生推荐 ----------- #-----------')
    print()
    item_topN_list = {} # 存储为每一个用户推荐的列表
    for each_user in cf.trainData:
        item_topN = cf.itemRecommend(each_user, k=3, N=5) # item的id和评分
        item_topN_list[each_user] = list(item_topN.keys()) # 只取对应的item的id
        if(each_user <= 5): # 输出id在1~5的用户推荐情况
            print('------ 长度5的电影序列item_topN_list的用户id：[{}] -----'.format(each_user))
            print(item_topN_list[each_user])
            print()
    
    # 真实值
    true_cf = CF('data/test.txt')
    true_cf.readData()
    true_cf.preprocessData()
    val_list = true_cf.trainData
    
    # 评估
    print('UserBasedCF的Precision值、Recall值、F_measure值分别为：{} '.format(FMeasure(user_topN_list, val_list)))
    print('ItemBasedCF的Precision值、Recall值、F_measure值分别为：{} '.format(FMeasure(item_topN_list, val_list)))

-----------# ----------- 基于【用户】为全部用户（打印id前5）产生推荐 ----------- #-----------

------ 长度5的电影序列user_topN_list的用户id：[1] -----
[28, 82, 127, 50, 4]

------ 长度5的电影序列user_topN_list的用户id：[2] -----
[7, 109, 118, 127, 117]

------ 长度5的电影序列user_topN_list的用户id：[3] -----
[6, 7, 93, 127, 28]

------ 长度5的电影序列user_topN_list的用户id：[4] -----
[53, 68, 85, 110, 118]

------ 长度5的电影序列user_topN_list的用户id：[5] -----
[50, 15, 25, 48, 60]

-----------# ----------- 基于【物品】为全部用户（打印id前5）产生推荐 ----------- #-----------

------ 长度5的电影序列item_topN_list的用户id：[1] -----
[127, 82, 21, 50, 90]

------ 长度5的电影序列item_topN_list的用户id：[2] -----
[127, 7, 98, 56, 137]

------ 长度5的电影序列item_topN_list的用户id：[3] -----
[7, 127, 98]

------ 长度5的电影序列item_topN_list的用户id：[4] -----
[13, 38, 21, 127, 98]

------ 长度5的电影序列item_topN_list的用户id：[5] -----
[88, 25, 99, 15, 82]

UserBasedCF的Precision值、Recall值、F_measure值分别为：(20.95, 17.39, 19.01) 
ItemBasedCF的Precision值、Recall值、F_measure值分别为：(19.42, 15.81, 17.43) 
